**<h3><center>EN2550 Assignment 3 - Object Counting On a Conveyor Belt</center></h3>**
*Name - Abeysinghe W.A.M.S.Y*
<br>
*Index no - 190014F*
<br>

All the imports are done at the beginning. 

In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

**Q1**. 